**Notebook Status:** Work in Progress

# Inverse Transform Sampling

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import dirichlet
from scipy.special import erf

#np.random.seed(4250)

In [ ]:
def normal_pdf(x, mu, sigma):
    return np.exp(-0.5*((x-mu)/sigma)**2)/(sigma*np.sqrt(2*np.pi))

In [ ]:
X = np.linspace(-10,10,500)
fig = px.area(x=X, y=normal_pdf(X, 0, 1))
fig.show()

Using this primitive probability measure, we may construct a composite measure as a convex combination

In [ ]:
n_gaussians = 5
mu_range = (-4, 4)
sigma_range = (0.25, 2)

In [ ]:
sampler = dirichlet([1]*n_gaussians)
convex_combination = sampler.rvs()[0]
rand_mus = np.random.uniform(*mu_range, n_gaussians)
rand_sigmas = np.random.uniform(*sigma_range, n_gaussians)

In [ ]:
gaussian_mixture = lambda x: (convex_combination[None,:] @ np.array([normal_pdf(x, rand_mus[i], rand_sigmas[i]) for i in range(n_gaussians)])).ravel()

In [ ]:
target_pdf = gaussian_mixture(X)
fig = px.area(x=X, y=target_pdf)
fig.show()

Here we see the individual (scaled) Gaussian components that are summed together to form the above target distribution we will approximate:

In [ ]:
components = dict(x=[], y=[], component=[])
for i in range(n_gaussians+1):
    components['x'].extend(X)
    if i:
        components['y'].extend(convex_combination[i-1]*normal_pdf(X, rand_mus[i-1], rand_sigmas[i-1]))
        components['component'].extend([i]*len(X))
    else:
        components['y'].extend(target_pdf)
        components['component'].extend(['mixture']*len(X))

fig = px.line(components, x='x', y='y', color='component')
fig.show()

## Sampling

Since we have access to our mixture parameters, we may integrate the PDF $f(x)$ to obtain the cumulative distribution function (CDF) $F(x)$ of our target distribution. 

$$F(x) = \int_{-\infty}^{\infty}f(x)\ dx.$$

Substituting $f(x) = \sum_{k=1}^{n}c_k\cdot p(x;\mu_k,\sigma_k)$ and applying elementary integration rules we may rewrite the target mixture CDF as

$$\sum_{k=1}^{n}c_k\cdot\int_{-\infty}^{\infty}p(x;\mu_k,\sigma_k)\ dx$$



In [ ]:
def normal_cdf(x, mu, sigma):
    return 0.5*(1+erf((x-mu)/(sigma*np.sqrt(2))))

fig = px.line(x=X, y=normal_cdf(X, 0, 1))
fig.show()

In [ ]:
mixture_cdf = lambda x: (convex_combination[None,:] @ np.array([normal_cdf(x, rand_mus[i], rand_sigmas[i]) for i in range(n_gaussians)])).ravel()
target_cdf = mixture_cdf(X)
fig = px.line(x=X, y=target_cdf)
fig.show()

Inverting this new CDF is not as straightforward analytically (though possible by manipulating the Maclaurin series representation of $\text{erf}^{-1}(x)$). For our purposes, we can approximate the inverse CDF by swapping the sampled x and y values of the target CDF and interpolating between these values. Given enough evenly-spaced samples of our target CDF, linear interpolation is sufficient and fast to compute compared to other interpolation methods, however we will consider other approaches later.

In [ ]:
interval = np.linspace(0, 1, 1000)
inverse_cdf = np.interp(interval, target_cdf, X)
fig = px.line(x=interval, y=inverse_cdf)
fig.show()

To sample from our target pdf, we implement inverse transform sampling.

Note that we exclude samples at the boundary of our interval $[-10, 10]$ to minimize artifacting in our sampler. In short, these artifacts emerge from the clipping process: whereas our target PDF is defined over the entire real line, our sampler is only defined over a closed interval.

In [ ]:
uniform_samples = np.random.random(10000)
target_samples = np.interp(uniform_samples, target_cdf, X)

# Remove boundary samples
target_samples = target_samples[np.abs(target_samples)<10]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=target_samples, histnorm='probability density', nbinsx=50, name='Histogram'))
fig.add_trace(go.Scatter(x=X, y=target_pdf, mode='lines', name='True PDF', line={'dash': 'dash'}))
fig.show()

# Kernel Density Estimation

In the previous section, we showed how to randomly sample from a known target distribution using inverse transform sampling (ITS). Unfortunately, for data we collect in the real world, we may not know a priori what distribution best fits our observations. For many data modelling problems, there are justified theoretical assumptions we can make, such as choosing a normal distribution to describe the distribution of adult heights across the world population, or fitting an exponential distribution to model the elapsed time between receiving two consecutive support tickets (in the context of a help desk). Such examples fall into the realm of **parametric** statistics. More generally however, if we want to describe data with minimal assumptions, we must resort to applying **non-parametric** methods. Note that while the generalized applicability of non-parametric methods is desirable, such benefits often come with trade-offs such as increased computational costs or error, which we discuss further below. 

In the following sections, we explore one such non-parametric method known as **kernel density estimation** (KDE) to construct an approximation of some target distribution, then show how we may implement ITS to to sample from this more general set of probability measures.


### What is a "Kernel"?

Although "kernel" refers to many different objects in mathematics, such as the kernel of a matrix (i.e., its nullspace), in the case of *kernel density estimation* this case refers to a "smoothing" function 
$K:X\to\mathbb{R}^{\ge 0}$ from some set $X$ to the real interval $[0, \infty)$. Intuitively, we can interpret such functions as assigning a non-negative weight to each element of $X$ Thus, probability measures may be understood as kernels whose total weight is normalized to sum (or integrate) to 1.

Another usage of the name "kernel", especially in the context of machine learning, optimization, functional analysis, operator theory, and other related domains, refers to a generalization of positive (semi)definite functions and operators (for example, positive definite matrices).

Let $X$ be a non-empty set. A symmetric binary function $K: X\times X\to\mathbb{R}$ is a positive semidefinite kernel if the following properties hold:

- For all $x,y\in X$, we have that $K(x, y) = K(y, x)$ (*symmetry*),
- For all $x_1\dots x_n\in X$, with $n$ a natural number, and all $a_1\dots a_n\in\mathbb{R}$, we have that $\sum_{i=1}^{n}\sum_{j=1}^{n}a_i a_jK(x_i,x_j)\ge 0$ (*positive semidefiniteness*).

We say a kernel function is *positive definite* when the equality $\sum_{i=1}^{n}\sum_{j=1}^{n}a_i a_jK(x_i,x_j)=0$ implies that $a_i=0$ for all $i$.

Although these two descriptions of kernels are seemingly unrelated, there exists a fundamental duality between probability measures and positive definite kernels, as formalized in Bochner's theorem. We will return to this connection in a following section; for now, we will focus on the former notion of a kernel as a positive measure in the context of nonparametric statistical analysis (skip section on *random Fourier features* for more details).

### Examples of Kernels for Density Estimation

In [ ]:
interval = np.linspace(-1, 1, 100)
compact_data = dict(
    uniform = 0*interval + 0.5,
    triangular = 1-np.abs(interval),
    biweight = (15/16)*(1-interval**2)**2,
    parabolic = 0.75*(1-interval**2),
)

fig = make_subplots(rows=1, cols=len(compact_data), shared_yaxes=True)
for i, key in enumerate(compact_data.keys()):
    fig.add_trace(
        go.Scatter(x=interval, y=compact_data[key], name=key, fill='tozeroy'),
        row=1, col=i+1
    )
fig.update_layout(title_text=r"Normalized Kernels with Compact Support: [-1, 1]", yaxis_title="Density")
fig.show()

In [ ]:
interval = np.linspace(-5, 5, 100)
inf_data = dict(
    gaussian = (1/np.sqrt(2*np.pi))*np.exp(-0.5*interval**2),
    sigmoid = 2/(np.pi*(np.exp(interval)+np.exp(-interval))),
)

fig = make_subplots(rows=1, cols=len(inf_data), shared_yaxes=True)
for i, key in enumerate(inf_data.keys()):
    fig.add_trace(
        go.Scatter(x=interval, y=inf_data[key], name=key, fill='tozeroy'),
        row=1, col=i+1
    )
fig.update_layout(title_text="Normalized Kernels with Infinite Support: (-∞, ∞)", yaxis_title='Density')
fig.show()

### KDE vs. Histograms

We may understand kernel density estimation as a generalization of histogram construction. To build a histogram, we first partition our sample space into a finite number of bins (i.e., compact connected disjoint subsets) that cover our space such that each sample we observe may be associated with exactly one bin. When observing data, we increment the weight of the bin associated with a particular sample, divided by the total number of samples in our dataset, if normalization is required. In other words, each data sample adds a unit of mass to a particular bin. At the end of this process, we are left with a discrete distribution that approximates the underlying distribution of our dataset. Since histograms are often applied to preliminary data visualization, their practical utility is typically limited to data of dimensions $n\le 3$, though this process is valid in any dimension.

The notion of a data sample adding "mass" to a histogram is preserved in kernel density estimation, however this mass is instead represented by a kernel centered at the data point. Intuitively, we want our mass (probability density) to concentrate around our observed data, regardless of where the data may lie in space. Thus, it is necessary to impose the additional condition of *translation-invariance* to our kernel so that $K(\mathbf{x},\mathbf{y})=K(\mathbf{x}+\mathbf{a},\mathbf{y}+\mathbf{a})$ for all
$\mathbf{x},\mathbf{y},\mathbf{a}\in\mathbb{R}^n$. To denote a translation-invariant kernel, we use the standard notation $K(\mathbf{x}-\mathbf{y})$ to emphasize that the kernel depends only on the *difference* between $\mathbf{x}$ and $\mathbf{y}$. Note that the examples of kernels provided in the previous section are all translation-invariant, and are thus appropriate for density estimation.

### Computing a KDE from Data

Let $X\subseteq\mathbb{R}^d$ with $\{x_1,\dots,x_n\}$ be a collection of independent and identically distributed (i.i.d.) samples drawn from from an unknown distribution $f$ over $X$, and let $K_h:X\times X\to\mathbb{R}$ be a normalized translation-invariant kernel with *bandwidth* $h\in\mathbb{R}^{+}$. Then, the kernel density estimator of $f$ at a point $x\in X$ is given by:
$$ 
\hat{f_h}(x) = \frac{1}{nh^d}\sum_{i=1}^{n}K(\frac{x-x_i}{h}).
$$
Note that the bandwidth $h$ serves to scale the width of our chosen kernel, allowing for finer control over the "spread" of density around each sample, such that values of $h$ closer to $0$ result in a "spikier" result (i.e., there is more high frequency information) and larger values of $h$ tend towards a "smoother" (low frequency) estimate. Below we show how the choice of bandwidth affects the KDE of our target gaussian mixture from before.

<hr/>

First, we define our kernel estimator to take as input a translation-invariant kernel, an array of i.i.d samples, an optional bandwidth parameter, and the point $x$ at which to estimate the unknown density. 

In [ ]:
def KDE(kernel_function, samples, x, bandwidth=1):
    scale = 1/(len(samples)*bandwidth)
    res = 0
    for sample in samples:
        res += scale*kernel_function((x-sample)/bandwidth)
    return res

Next we draw i.i.d. samples from our target distribution, simulating some ideal observation process.

In [ ]:
n_observations = 100
samples = np.interp(np.random.random(n_observations), target_cdf, X)

To illustrate the effect of various choices of bandwidth, we fix a particular kernel and dataset for our estimator.

In [ ]:
parabolic_kernel = lambda x: np.array([0 if abs(v) > 1 else 0.75*(1-v**2) for v in x])
gaussian_kernel = lambda x: (1/np.sqrt(2*np.pi))*np.exp(-0.5*x**2)

bandwidths = [0.25, 0.5, 1, 2]

In [ ]:
density_estimates = dict(
    parabolic = {h: KDE(parabolic_kernel, samples, X, bandwidth=h) for h in bandwidths},
    gaussian = {h: KDE(gaussian_kernel, samples, X, bandwidth=h) for h in bandwidths},
)

fig = make_subplots(rows=2, 
                    cols=len(bandwidths), 
                    shared_yaxes=True,
                    subplot_titles=[f'BW: {bandwidths[i]}' for i in range(len(bandwidths))]
)

for i, kernel in enumerate(density_estimates.keys()):
    for j, h in enumerate(density_estimates[kernel].keys()):
        fig.add_trace(
            go.Scatter(
                x=X, 
                y=density_estimates[kernel][h], 
                name=f"Estimate ({kernel})", 
                fill='tozeroy', 
                line={'color': ['blue', 'green'][i]},
                legendgroup=0,
                showlegend=not bool(j),
            ), 
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scatter(
                x=X, 
                y=target_pdf, 
                name="True Density",
                line={'dash': 'dash', 'color': 'red'},
                legendgroup=1,
                showlegend=not bool(i+j),
            ), 
        row=i+1, col=j+1
        )

fig.update_layout(title='KDE Result vs. Bandwidth', yaxis_title="Density")
fig.show()

### Variable Bandwidth KDE

See [Ref 1](https://journals.sagepub.com/doi/pdf/10.1177/1536867X0300300203) and [Ref 2](https://perso.univ-rennes2.fr/system/files/users/rouviere_l/hvariable-fin.pdf) for details.

In [ ]:
def VBKDE(kernel_function, samples, x, iters=1, bandwidth=1):
    weights = np.ones(len(samples)) 
    scale = 1/(len(samples)*bandwidth)
    for i in range(iters):
        sample_density = 0
        for j, val in enumerate(samples):
            sample_density += (scale/weights[j])*kernel_function((samples-val)/(bandwidth*weights[j]))
        #gmean = np.prod(sample_density)**(1.0/len(samples))  # unstable
        gmean = np.exp(np.sum(np.log(np.abs(sample_density)))/len(samples)) 
        weights = (gmean/sample_density)**0.5
    res = 0
    for i, val in enumerate(samples):
        res += (scale/weights[i])*kernel_function((x-val)/(bandwidth*weights[i]))
    return res

In [ ]:
n_iters = 5
density_estimates = dict(
    parabolic = {h: VBKDE(parabolic_kernel, samples, X, iters=n_iters, bandwidth=h) for h in bandwidths},
    gaussian = {h: VBKDE(gaussian_kernel, samples, X, iters=n_iters, bandwidth=h) for h in bandwidths},
)

fig = make_subplots(rows=2, 
                    cols=len(bandwidths), 
                    shared_yaxes=True,
                    subplot_titles=[f'BW: {bandwidths[i]}' for i in range(len(bandwidths))]
)

for i, kernel in enumerate(density_estimates.keys()):
    for j, h in enumerate(density_estimates[kernel].keys()):
        fig.add_trace(
            go.Scatter(
                x=X, 
                y=density_estimates[kernel][h], 
                name=f"Estimate ({kernel})", 
                fill='tozeroy', 
                line={'color': ['blue', 'green'][i]},
                legendgroup=0,
                showlegend=not bool(j),
            ), 
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scatter(
                x=X, 
                y=target_pdf, 
                name="True Density",
                line={'dash': 'dash', 'color': 'red'},
                legendgroup=1,
                showlegend=not bool(i+j),
            ), 
        row=i+1, col=j+1
        )

fig.update_layout(title='Variable Bandwidth KDE (Sample Point) w/ Initial Bandwidth', yaxis_title="Density")
fig.show()

# Monte Carlo Integration

TBD

# Random Fourier Features 

Per Bochner's theorem: Suppose $f$ is a normalized positive definite kernel, where normalization means that $f(0)=1$. Then, there exists a unique probability distribution $p$ such that $f(x)=\int_{-\infty}^{\infty}p(\omega)e^{i\omega x}\ d\omega$. In other words, $f$ and $p$ are related to each other by the Fourier transform.

Rahimi and Recht ([ref](https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf)) detail how we may construct an estimator of $f$ through the use of *random Fourier features* (RFF) -- this process reduces the computational cost of evaluating kernel functions, at the expense of accuracy (though error is inversely proportional to the number of features). Their approach is useful in the context of support vector machines, density estimation, and other kernel-based methods, especially when the explicit computation of a kernel function is expensive (more on this further below).

In short, we can rewrite the expression $\int_{-\infty}^{\infty}p(\omega)e^{i\omega x}\ d\omega$ using the law of the unconscious statistician as the expectation $\mathbb{E}_\omega[e^{i\omega x}]$. Note that this is also the definition of a *characteristic function* of a probability distribution. Then, an unbiased estimator of $f$ may be constructed via Monte Carlo integration:

$$f(x)=\mathbb{E}_\omega[e^{i\omega x}]\approx\frac{1}{N}\sum_{k=1}^{N}e^{i\omega x},$$

where $N$ is the number of samples $\omega$ drawn uniformly from $p$. Because $e^{ix}$ is a complex-valued function, when working with real kernels we can instead use the estimator $\frac{1}{N}\sum_{k=1}^{N}\cos({\omega x}),$ since $e^{ix}=\cos(x)+i\sin(x)$, per Euler. 

Below we show some examples of univariate kernel functions over $\mathbb{R}$ estimated using this method.

### Examples

Use the sliders in the following examples to see how each estimator converges to the true kernel function as the number of samples increases.

In [ ]:
# Define simulation variables
max_samples = 2000
srange = np.arange(0, max_samples+1, 5)
srange[0] = 1
n_frames = len(srange)

#### Example 1: Gaussian Distribution
Here we sample from the standard Gaussian distribution, whose Fourier transform is another Gaussian function.

In [ ]:
# Sample from standard Gaussian
samples = np.random.normal(0, 1, max_samples)

fig = go.Figure()
for f in range(n_frames):
    fig.add_trace(
        go.Scatter(
            x = X,
            y = (1/len(samples[:srange[f]])) * np.sum(np.cos(np.outer(samples[:srange[f]], X)), axis=0),
            name = 'estimate',
            fill='tozeroy',
            visible = False if f else True,
        )
    )

steps = []
for i in range(n_frames):
    step = dict(
        method="restyle",
        args=[{"visible": [False] * n_frames + [True]}],
        label=str(srange[i]),
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Num. Samples: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title='Gaussian Kernel Approximation',
    yaxis=dict(title='y'),
    xaxis=dict(title='x'),
)

fig.add_trace(go.Scatter(x=X, y=np.exp(-0.5*X**2), line={'dash':'dash'}, name='true kernel'))
fig.show()

#### Example 2: Uniform Distribution

Here we sample from a uniform distribution over the interval $[-\pi,\pi]$, whose Fourier transform is $\operatorname{sinc}(x)=\frac{\sin({\pi x})}{\pi x}$.

In [ ]:
# Sample from uniform distribution over circle
samples = np.random.uniform(-np.pi, np.pi, max_samples)

fig = go.Figure()
for f in range(n_frames):
    fig.add_trace(
        go.Scatter(
            x = X,
            y = (1/len(samples[:srange[f]])) * np.sum(np.cos(np.outer(samples[:srange[f]], X)), axis=0),
            name = 'estimate',
            fill='tozeroy',
            visible = False if f else True,
        )
    )

steps = []
for i in range(n_frames):
    step = dict(
        method="restyle",
        args=[{"visible": [False] * n_frames + [True]}],
        label=str(srange[i]),
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Num. Samples: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title='Sinc Kernel Approximation',
    yaxis=dict(title='y'),
    xaxis=dict(title='x'),
)

fig.add_trace(go.Scatter(x=X, y=np.sin(np.pi*X)/(np.pi*X), line={'dash':'dash'}, name='true function'))
fig.show()

#### Example 3: Parabolic Distribution

Here we sample from a parabolic distribution using ITS and apply the same procedure as above. Note that the Fourier transform of this distribution does not have a standard name (as far as I am aware), though it can be solved for in closed form.

In [ ]:
# Sample from normalized parabolic distribution using inverse transform sampling.
unit = np.linspace(-1, 1, 200)
cdf = -0.25*(unit**3-3*unit-2)
ppf = lambda x: np.interp(x, cdf, unit)
rand = np.random.random(max_samples)
samples = ppf(rand)

fig = go.Figure()
for f in range(n_frames):
    fig.add_trace(
        go.Scatter(
            x = X,
            y = (1/len(samples[:srange[f]])) * \
                np.sum(np.cos(np.outer(np.pi*samples[:srange[f]], X)), axis=0),  # (accounts for pi stretch factor)
            name = 'estimate',
            fill='tozeroy',
            visible = False if f else True,
        )
    )

steps = []
for i in range(n_frames):
    step = dict(
        method="restyle",
        args=[{"visible": [False] * n_frames + [True]}],
        label=str(srange[i]),
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Num. Samples: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title='Parabolic Kernel Approximation',
    yaxis=dict(title='y'),
    xaxis=dict(title='x'),
)

fig.add_trace(go.Scatter(x=X, y=3*(np.sin(np.pi*X)-np.pi*X*np.cos(np.pi*X))/(np.pi*X)**3, line={'dash':'dash'}, name='true function'))
fig.show()

# TBD